# Data voorbereiden

Dit Notebook moet gebruikt worden vóór het trainen van de beeldherkenning (Beeldherkenning.ipynb).

In de volgende code worden data schoongemaakt en samengevoegd. De volgende opties kunnen ingesteld worden:
1. Welke folders samengevoegd moeten worden
2. Welke folders hernoemd moeten worden
3. Welke folders overgeslagen/genegeerd moeten worden

Het resultaat kost niet zoveel data, omdat er alleen links naar de originele (afbeeldings)bestanden gecreëert worden. De bestanden worden dus **niet** gekopieerd.

## Imports

In [1]:
import csv
import random
from pathlib import Path
from joblib import Parallel, delayed
from PIL import Image
from tqdm import tqdm


## Instellingen

Pas deze variabelen aan voor de situatie.

- data_path: pad naar de folder waar het resultaat (= de gecombineerde dataset) geplaatst wordt. Het resultaat zal als volgt zijn:
  ```
    ├── data_path
    │   ├── Klasse A
    │   ├── Klasse B
    │   └── Klasse C
  ```
- label_translation_path: naam van het bestand waarin staat welke folders hernoemd moeten worden
- label_combination_path: naam van het bestand waarin staat welke folders gecombineerd moeten worden
- delimiter: scheidingsteken in de .csv bestanden
- encoding: encoding van de .csv bestanden. Deze moet aangepast worden als de bestanden verkeerd worden ingelezen (bv. verkeerde leestekens)
- import_paths: paden naar folders waar data staan die gecombineerd moeten worden tot één dataset. In elke folder moeten folders staan per klasse. Voorbeeld:
  ```
    ├── import_paths[0]
    │   ├── Klasse A
    │   └── Klasse B
    ├── import_paths[1]
    │   ├── Klasse B
    │   └── Klasse C
    └── etc...
  ```
  NB: de import_paths kunnen in verschillende directories staan.


In [3]:
output_folder_name = 'dataset'

label_combination_path = ''  # '/data/mothRecognition/resources/pseudospecies_beeldherkenning_V20230906.csv'
sub_label_index = 0
combo_label_index = 2

label_translation_path = '/data/mothRecognition/resources/EngelseNamenCompleet.csv'
old_label_index = 0
new_label_index = 1

delimiter = ';'
encoding = 'windows-1252'

char_replacements = {'/': '-',
                     '_': '-',
                     '.': ''}

import_paths = [Path('')
    Path('/data/mothRecognition/data/IRecord/moths_images'),
                Path('/data/mothRecognition/data/raam'),
                Path('/data/mothRecognition/data/telmee/photos'),
                Path('/data/mothRecognition/data/lepiforum/macro'),
                Path('/data/mothRecognition/data/lepiforum/micro'),
                Path('/data/mothRecognition/data/meetnet'),
                Path('/data/mothRecognition/data/gbif/micro_photos_gbif_15'),
                Path('/data/mothRecognition/data/gbif/micro_photos_gbif_16'),
                Path('/data/mothRecognition/data/gbif/micro_photos_gbif_17'),
                Path('/data/mothRecognition/data/gbif/micro_photos_gbif_18'),
                Path('/data/mothRecognition/data/gbif/micro_photos_gbif_19'),
                Path('/data/mothRecognition/data/gbif/micro_photos_gbif_20'),
                Path('/data/mothRecognition/data/gbif/micro_photos_gbif_21_22')]

n_jobs = 10


## Voorbereiding

Laad de bestanden met informatie over de klassen (label_translation_path & label_combination_path).

### Labels combineren

In [4]:
combo_dict = {}
if label_combination_path != '':
    with open(label_combination_path, 'r', encoding=encoding) as combination_file:
        reader = csv.reader(combination_file, delimiter=delimiter)
        headers = next(reader)
        print(f'Kolomnamen in {label_combination_path}: {headers}')
        print(f"sub_label_index points to '{headers[sub_label_index]}'")
        print(f"combo_label_index points to '{headers[combo_label_index]}'")
    
        for row in reader:
            if row[sub_label_index] != '':
                sub_label = row[sub_label_index].lower()
                combo_dict[sub_label] = row[combo_label_index].lower()
                for old_char, new_char in char_replacements.items():
                    combo_dict[sub_label] = combo_dict[sub_label].replace(old_char, new_char)
    
combo_labels = list(set(combo_dict.values()))


In [5]:
if len(combo_dict) > 0:
    print(f'Er zullen {len(combo_dict)} labels gecombineerd worden tot {len(combo_labels)} labels (verschil = {len(combo_dict) - len(combo_labels)})')
    example_labels = random.sample(sorted(combo_dict), 3)
    print(f'Voorbeelden van originele labels: {example_labels}')
    print(f'Bijbehorende combinatie labels: {[combo_dict[label]for label in example_labels]}')


### Labels hernoemen

In [6]:
alt_labels = {}
if label_translation_path != '':
    with open(label_translation_path, 'r') as translation_file:
        reader = csv.reader(translation_file, delimiter=delimiter)
        headers = next(reader)
        print(f'Kolomnamen in {label_translation_path}: {headers}')
        print(f"old_label_index points to '{headers[old_label_index]}'")
        print(f"new_label_index points to '{headers[new_label_index]}'")
    
        for row in reader:
            if row[old_label_index] != '':
                alt_labels[row[old_label_index].lower()] = row[new_label_index].lower()


Kolomnamen in /data/mothRecognition/resources/EngelseNamenCompleet.csv: ['Engelse naam', 'Wetenschappelijke naam']
old_label_index points to 'Engelse naam'
new_label_index points to 'Wetenschappelijke naam'


In [7]:
if len(alt_labels) > 0:
    print(f'Aantal labels die vertaald/hernoemd gaan worden: {len(alt_labels)}')
    example_labels = random.sample(sorted(alt_labels), 3)
    print(f'Voorbeelden van originele labels: {example_labels}')
    print(f'Bijbehorende hernoemde labels: {[alt_labels[label]for label in example_labels]}')


Aantal labels die vertaald/hernoemd gaan worden: 1158
Voorbeelden van originele labels: ['chocolate-tip', 'beautiful china-mark', 'cistus forester']
Bijbehorende hernoemde labels: ['clostera curtula', 'nymphula nitidulata', 'adscita geryon']


## Data valideren

Voordat alle bestanden uit de folders tot één dataset gecombineerd worden, worden de kapotte bestanden eruit gefilterd. Het kan namelijk best voorkomen dat er afbeeldingen zijn die niet ingelezen kunnen worden. Die moeten niet in de uiteindelijke dataset zitten.

In [8]:
output_path = (Path('') / output_folder_name).absolute()
print(f'Pad naar gecombineerde dataset: {output_path}')

Pad naar gecombineerde dataset: /data/mothRecognition/notebooks/datasetNotCombined


In [9]:
paths_to_validate = []

for path in import_paths:
    for class_path in path.iterdir():
        if not class_path.is_dir():
            continue
        for file_path in class_path.iterdir():
            paths_to_validate.append(file_path.absolute())

print(f'Aantal bestanden die gevalideerd gaan worden: {len(paths_to_validate)}')


Aantal bestanden die gevalideerd gaan worden: 1040047


Hieronder staat de functie die gebruikt zal worden om de bestanden te valideren. Standaard word er geprobeerd het bestand te openen en om te zetten naar RGB. Als dat lukt, dan is het bestand in goede staat. Je kan dit aanpassen in het `try` blok, als je op een andere manier de bestanden wilt valideren.

In [10]:
def validate_path(file_path):
    try:
        image = Image.open(file_path).convert("RGB")
        exif = image.getexif()
    except Exception as e:
        return file_path, False        
    else:
        return file_path, True


In [11]:
validation_results = Parallel(n_jobs=n_jobs)(
    delayed(validate_path)(p) for p in tqdm(paths_to_validate)
)

valid_paths = [p for p, valid in validation_results if valid]


 58%|███████████████████████████████████████████████████████████████████████████████████████████████▎                                                                   | 608300/1040047 [05:50<03:07, 2305.70it/s]/home/farfalla/miniconda3/envs/ml/lib/python3.11/site-packages/PIL/TiffImagePlugin.py:866: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
 59%|███████████████████████████████████████████████████████████████████████████████████████████████▍                                                                   | 609260/1040047 [05:50<02:51, 2514.32it/s]/home/farfalla/miniconda3/envs/ml/lib/python3.11/site-packages/PIL/TiffImagePlugin.py:866: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
 59%|████████████████████████████████████████████████████████████████████████████████████████████████▏                                                                  | 614060/1040047 [05:52<

In [12]:
print(f'{len(valid_paths)} bestandspaden zijn valide, en {len(paths_to_validate) - len(valid_paths)} paden zijn afgekeurd.')

1039684 bestandspaden zijn valide, en 363 paden zijn afgekeurd.


## Data combineren naar één dataset

Nu alle data gevalideerd (of afgekeurd) is, kunnen de verschillende databronnen gecombineerd worden tot één dataset.

De verschillende stappen zijn:
1. Hernoemen van labels (staan in `alt_labels`)
2. Filteren van klasses hoger dan soortniveau die *niet* in `combo_labels` voorkomen
    - Alle labels op soortniveau worden naar '\[Genus\] spec' geformatteerd
4. Opschonen van de labels m.b.v. `char_replacements`
5. Pseudo-/combinatiesoorten wegfilteren als ze *niet* in `combo_labels` voorkomen
6. Soorten herlabellen als pseudo-/combinatiesoort als ze *wel* in `combo_labels` voorkomen
7. Subsoorten wegfilteren

Het resultaat is dat er alleen labels op soortniveau zijn, tenzij er een uitzondering voor een pseudo-/combinatiesoort was. Soorten met verschillende namen zijn samengevoegd onder één label.

In [13]:
n_pseudospecies_images = 0
n_genus_images = 0
n_subspecies_images = 0

for i, path in tqdm(enumerate(valid_paths)):    
    class_name = path.parent.name.lower()
    
    # Replace alternative labels with real labels
    if class_name in alt_labels:
        class_name = alt_labels[class_name]

    # Combine sp., sp, (...), and only genus to spec
    split = class_name.split()
    if len(split) == 1 or split[1][0] == '(' or split[1] == 'sp.' or split[1] == 'sp' or split[1].startswith('agg'):
        class_name = split[0] + ' spec'
        # Only keep this label if it is specified in combo_labels
        if class_name not in combo_labels:
            n_genus_images += 1
            continue
    
    # Replace certain characters
    for old_char, new_char in char_replacements.items():
        class_name = class_name.replace(old_char, new_char)

    # Remove non-allowed pseudospecies
    split = class_name.split()[1].split("-")
    if class_name not in combo_labels and len(split) > 1 and len(split[0]) > 1:
        n_pseudospecies_images += 1
        continue

    # Combine the combo species
    if class_name in combo_dict:
        class_name = combo_dict[class_name]

    # Remove subspecies
    split = class_name.split(" f ")
    if len(split) > 1:
        n_subspecies_images += 1
        continue
    
    symbolic_link_path = (output_path / class_name / f"{i:07}").with_suffix(
        path.suffix
    )
    symbolic_link_path.parent.mkdir(exist_ok=True, parents=True)
    symbolic_link_path.unlink(missing_ok=True)
    symbolic_link_path.symlink_to(path)

skipped_images = len(paths_to_validate) - len(valid_paths) + n_pseudospecies_images + n_genus_images + n_subspecies_images

print(f"\n{len(paths_to_validate) - skipped_images} links naar de originele bestanden zijn geplaatst in {str(output_path)}")
print(f"{skipped_images} bestanden zijn overgeslagen, waarvan:")
print(f"  {len(paths_to_validate) - len(valid_paths)} niet door de validatie zijn gekomen")
print(f"  {n_pseudospecies_images} pseudo-/combinatiesoorten zijn die niet uitgezonderd waren")
print(f"  {n_genus_images} niet op soortniveau, maar op genusniveau gedefinieerd waren")
print(f"  {n_subspecies_images} niet op soortniveau, subsoortniveau gedefinieerd waren")


1039684it [00:24, 42677.18it/s]


1037930 links naar de originele bestanden zijn geplaatst in /data/mothRecognition/notebooks/datasetNotCombined
2117 bestanden zijn overgeslagen, waarvan:
  363 niet door de validatie zijn gekomen
  1667 pseudo-/combinatiesoorten zijn die niet uitgezonderd waren
  70 niet op soortniveau, maar op genusniveau gedefinieerd waren
  17 niet op soortniveau, subsoortniveau gedefinieerd waren


## Conclusie

Het resultaat van dit notebook is een gecombineerde en opgeschoonde dataset die gebruikt kan worden voor o.a. machine learning. De volgende stap is om de dataset te gebruiken voor het andere notebook (BeeldherkenningTrainen.ipynb).